In [29]:
import _init_paths

import torch
import os
from model.faster_rcnn.vgg16 import vgg16
from model.utils.config import cfg

## load model

In [30]:
imdb_name = "KAIST_train"
class_agnostic = False
lr=0.001
imdb_classes =  ('__background__',  # always index 0
                          'person',
                          'people','cyclist'
                         )
   
#-----param---------------
params = []
for key, value in dict(fasterRCNN.named_parameters()).items():
    if value.requires_grad:
        if 'bias' in key:
            params += [{'params':[value],'lr':lr*(cfg.TRAIN.DOUBLE_BIAS + 1), \
                    'weight_decay': cfg.TRAIN.BIAS_DECAY and cfg.TRAIN.WEIGHT_DECAY or 0}]
        else:
            params += [{'params':[value],'lr':lr, 'weight_decay': cfg.TRAIN.WEIGHT_DECAY}]
#--------------------


In [75]:
def load_fasterRCNN_model(path):

    fasterRCNN = vgg16(imdb_classes, pretrained=True, class_agnostic=class_agnostic)
    fasterRCNN.create_architecture()
    fasterRCNN.cuda()

    optimizer = torch.optim.SGD(params, momentum=cfg.TRAIN.MOMENTUM)


    checkpoint = torch.load(path)  
    #session = checkpoint['session']
    #start_epoch = checkpoint['epoch']

    fasterRCNN.load_state_dict(checkpoint['model']) 
    optimizer.load_state_dict(checkpoint['optimizer'])
    #lr = optimizer.param_groups[0]['lr']


    if 'pooling_mode' in checkpoint.keys():
          POOLING_MODE = checkpoint['pooling_mode']
    
    return fasterRCNN

In [48]:
POOLING_MODE

'align'

In [79]:
def avgWeight(model_list):
    model_tmp=[None] * parties
    #optims_tmp=[None] * parties

    for idx, my_model in enumerate(model_list):
        
        model_tmp[idx] = my_model.state_dict()


    for key in model_tmp[0]:    
        #print(key)
        model_sum = 0

        for model_tmp_content in model_tmp:      
            
            model_sum += model_tmp_content[key]
            #print(model_tmp_content[key])
        for i in range(len(model_tmp)):
            #print("model_sum={}".format(model_sum))
            #print("len:{}".format(len(model_tmp)))
            model_avg = model_sum/len(model_tmp)
            #print("model_avg={}".format(model_avg))
            model_tmp[i][key] = model_sum/len(model_tmp)
    for i in range(len(model_list)):    
        model_list[i].load_state_dict(model_tmp[i])
        #optims_tmp[i] = Optims(workers, optim=optim.SGD(params=model_list[i].parameters(),lr=args.lr, momentum = args.momentum,weight_decay=args.weight_decay))
        #optims_tmp[i] = Optims(workers, optim=optim.Adam(params=model_list[i].parameters(),lr=args.lr))
    return model_list  #, optims_tmp
    

In [89]:
path_checkpoint_campus ='models/vgg16/KAIST/faster_rcnn_KAIST_campus_1_2_718.pth'
path_checkpoint_road ='models/vgg16/KAIST/faster_rcnn_KAIST_road_1_2_513.pth'
path_checkpoint_downtown ='models/vgg16/KAIST/faster_rcnn_KAIST_downtown_1_2_566.pth'
parties = 3

model_list=[None] * parties
model_list[0] = load_fasterRCNN_model(path_checkpoint_campus)
model_list[1] = load_fasterRCNN_model(path_checkpoint_road)
model_list[2] = load_fasterRCNN_model(path_checkpoint_downtown)


Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth


In [90]:
model_avg = avgWeight(model_list)

In [91]:
from model.utils.net_utils import save_checkpoint
output_dir = 'models/vgg16/KAIST'
save_name = os.path.join(output_dir, 'faster_rcnn_KAIST_AVG_{}.pth'.format(1))
save_checkpoint({
  'session': 1,
  'epoch': 1,
  'model':  model_avg[0].state_dict(),
  'optimizer': optimizer.state_dict(),
  'pooling_mode': POOLING_MODE,
  'class_agnostic': class_agnostic,
}, save_name)
